In [1]:
!pip install numpy==1.16.1


In [1]:
import numpy
from numpy import array
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Dropout
#from tensorflow.keras.layers.embeddings import Embedding
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model
import re
import numpy as np
from nltk.tokenize import word_tokenize
import nltk

# fix random seed for reproducibility
numpy.random.seed(7)

In [3]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)

In [4]:
print(X_train[6],y_train[6])

[1, 2, 365, 1234, 5, 1156, 354, 11, 14, 2, 2, 7, 1016, 2, 2, 356, 44, 4, 1349, 500, 746, 5, 200, 4, 4132, 11, 2, 2, 1117, 1831, 2, 5, 4831, 26, 6, 2, 4183, 17, 369, 37, 215, 1345, 143, 2, 5, 1838, 8, 1974, 15, 36, 119, 257, 85, 52, 486, 9, 6, 2, 2, 63, 271, 6, 196, 96, 949, 4121, 4, 2, 7, 4, 2212, 2436, 819, 63, 47, 77, 2, 180, 6, 227, 11, 94, 2494, 2, 13, 423, 4, 168, 7, 4, 22, 5, 89, 665, 71, 270, 56, 5, 13, 197, 12, 161, 2, 99, 76, 23, 2, 7, 419, 665, 40, 91, 85, 108, 7, 4, 2084, 5, 4773, 81, 55, 52, 1901] 1


In [5]:
word2id = imdb.get_word_index()
id2word = {i:word for word , i in word2id.items()}

In [6]:
print([id2word.get(i, ' ') for i in X_train[6]])

['the', 'and', 'full', 'involving', 'to', 'impressive', 'boring', 'this', 'as', 'and', 'and', 'br', 'villain', 'and', 'and', 'need', 'has', 'of', 'costumes', 'b', 'message', 'to', 'may', 'of', 'props', 'this', 'and', 'and', 'concept', 'issue', 'and', 'to', "god's", 'he', 'is', 'and', 'unfolds', 'movie', 'women', 'like', "isn't", 'surely', "i'm", 'and', 'to', 'toward', 'in', "here's", 'for', 'from', 'did', 'having', 'because', 'very', 'quality', 'it', 'is', 'and', 'and', 'really', 'book', 'is', 'both', 'too', 'worked', 'carl', 'of', 'and', 'br', 'of', 'reviewer', 'closer', 'figure', 'really', 'there', 'will', 'and', 'things', 'is', 'far', 'this', 'make', 'mistakes', 'and', 'was', "couldn't", 'of', 'few', 'br', 'of', 'you', 'to', "don't", 'female', 'than', 'place', 'she', 'to', 'was', 'between', 'that', 'nothing', 'and', 'movies', 'get', 'are', 'and', 'br', 'yes', 'female', 'just', 'its', 'because', 'many', 'br', 'of', 'overly', 'to', 'descent', 'people', 'time', 'very', 'bland']


In [7]:
print(y_train[6])

1


In [8]:
print('Maximum review length: {}'.format(
len(max((X_train + X_test), key=len))))

Maximum review length: 2697


In [9]:
print('Minimum review length: {}'.format(
len(min((X_train + X_test), key=len))))

Minimum review length: 70


In [10]:
max_review_length = 500

In [11]:
'''In order to feed this data into our RNN, all input documents must have the same length. We will limit the maximum review length to maximum words by truncating 
longer reviews and padding shorter reviews with a null value (0). We can accomplish this task using the pad_sequences() function in Keras. Here, setting max_review_length 
to 500.'''

X_train = sequence.pad_sequences(X_train,maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test,maxlen=max_review_length)

In [12]:
top_words = 5000

In [13]:
embedding_vector_lenth = 32
model = Sequential()
model.add(Embedding(top_words,embedding_vector_lenth,input_length=max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
dropout (Dropout)            (None, 500, 32)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
dropout (Dropout)            (None, 500, 32)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 344s 14ms/sample - loss: 0.4484 - accuracy: 0.7882 - val_loss: 0.3195 - val_accuracy: 0.8680
Epoch 2/10
25000/25000 [==============================] - 362s 14ms/sample - loss: 0.2709 - accuracy: 0.8948 - val_loss: 0.2927 - val_accuracy: 0.8777
Epoch 3/10
25000/25000 [==============================] - 383s 15ms/sample - loss: 0.2494 - accuracy: 0.9005 - val_loss: 0.3241 - val_accuracy: 0.8666
Epoch 4/10
25000/25000 [==============================] - 411s 16ms/sample - loss: 0.2174 - accuracy: 0.9180 - val_loss: 0.3055 - val_accuracy: 0.8750
Epoch 5/10
25000/25000 [==============================] - 449s 18ms/sample - loss: 0.2503 - accuracy: 0.8962 - val_loss: 0.3958 - val_accuracy: 0.8540
Epoch 6/10
25000/25000 [==============================] - 495s 20ms/sample - loss: 0.1889 - accuracy: 0.9284 - val_loss: 0.3270 - val_accuracy: 0.8654
Epoch 7/10
25000/25000 [====================

In [17]:
model.save("sentiment.h5")

In [2]:
model = load_model('sentiment.h5')

In [12]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 86.69%
